In [ ]:
from datetime import datetime as dt
import time
from selenium import webdriver
from selenium.webdriver.support.select import Select
import pandas as pd
#import requests
#from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import pandas as pd

def data_scrapingZ(driver, original_price_list, link_booking_list):
    # Original Price
    try:
        prices = driver.find_elements(By.XPATH, '//span[@data-testid="price-and-discounted-price"]')
        for price in prices:
            try:
                original_price_list.append(price.text)
            except StaleElementReferenceException:
                original_price_list.append('N/A')
    except NoSuchElementException:
        return None

    # link booking
    try:
        link_booking_elements = driver.find_elements(By.XPATH, '//div[@data-testid="property-card"]')
        for link_element in link_booking_elements:
            try:
                link = link_element.find_element(By.XPATH, './/a[@href]')
                link_url = link.get_attribute('href')
                if link_url:
                    link_booking_list.append(link_url)
            except (NoSuchElementException, StaleElementReferenceException):
                pass
    except NoSuchElementException:
        return None

    return original_price_list, link_booking_list

def crawl_booking_dot_com(base_url):
    original_price_list, link_booking_list = ([] for _ in range(2))
    driver.get(base_url)

    for page_number in range(0, 20):
        try:
            original_price_list1, link_booking_list1 = data_scrapingZ(driver, original_price_list, link_booking_list)
            print("Link ", len(link_booking_list1))
            print("Giá ", len(original_price_list1))
        except (StaleElementReferenceException, NoSuchElementException):
            print("Exception occurred. Retrying...")
            continue

        # Kiểm tra xem có trang kế tiếp không
        next_button = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, "//button[@aria-label='Trang sau']"))
        )

        if next_button:
            page_number += 1
        else:
            print("Đã crawl hết các trang.")
            break

        url = f"{base_url}&offset={page_number * 25}"
        print("so trang", page_number)
        driver.get(url)

    print("Toi day chua")

    # Tạo DataFrame từ danh sách và lưu vào file CSV
    if len(original_price_list1) == len(link_booking_list1):
        df = pd.DataFrame({'price': list(original_price_list1), 'Link': list(link_booking_list1)})
        df.to_csv(r'C:\Users\thinh\OneDrive\Máy tính\LinkBookingDaNang.csv')
    else:
        print(original_price_list1)
        print(link_booking_list1)
        print(len(original_price_list1))


In [ ]:
linkZ = 'https://www.booking.com/searchresults.vi.html?ss=%C4%90%C3%A0+N%E1%BA%B5ng%2C+Th%C3%A0nh+ph%E1%BB%91+%C4%90%C3%A0+N%E1%BA%B5ng%2C+Vi%E1%BB%87t+Nam&ssne=Vu%CC%83ng+Ta%CC%80u&ssne_untouched=Vu%CC%83ng+Ta%CC%80u&label=gen173nr-1FCAEoggI46AdIM1gEaPQBiAEBmAEquAEXyAEM2AEB6AEB-AELiAIBqAIDuALo9b-vBsACAdICJDZmYWViZGE3LTg3OTUtNDcxZC1hZGM2LTcyNmQ4YjFiZGM4M9gCBuACAQ&sid=eceff66f0c492ef7b95dec3d8b86b680&aid=304142&lang=vi&sb=1&src_elem=sb&src=searchresults&dest_id=-3712125&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=vi&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=bffc5c8e4316000d&ac_meta=GhBiZmZjNWM4ZTQzMTYwMDBkIAAoATICdmk6AkRhQABKAFAA&checkin=2024-03-13&checkout=2024-03-16&group_adults=1&no_rooms=1&group_children=0'
driver = webdriver.Chrome()
crawl_booking_dot_com(linkZ)